<a href="https://colab.research.google.com/github/micheal-el07/coursera_data_science/blob/main/course_3_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import requests
from datetime import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

--2023-10-21 12:28:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45 [text/csv]
Saving to: ‘exchange_rate.csv’

exchange_rate.csv   100%[===================>]      45  --.-KB/s    in 0s      

2023-10-21 12:28:54 (15.8 MB/s) - ‘exchange_rate.csv’ saved [45/45]



In [6]:
# Code for ETL operations on Country-GDP data
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

output_path = 'market_cap.csv'
table_attribs = ["Name", "MC_USD_Billion"]
path = 'exchange_rate.csv'

table_name = 'Largest_banks'
db_name = 'Banks.db'


# Importing the required libraries

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%m-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('./code_log.txt', "a") as f:
      f.write(timestamp + ":" + message + '\n')

def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    for row in rows:
      col = row.find_all('td')
      if len(col) != 0:
        data_dict = {"Name": col[1].find_all('a')[1]['title'],"MC_USD_Billion": float(col[2].contents[0].strip())}
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df, df1], ignore_index=True)

    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    edf = pd.read_csv('exchange_rate.csv')
    exchange_rate_dict = edf.set_index('Currency').to_dict()['Rate']
    exchange_rate_dict

    df['MC_GBP_Billion'] = [np.round(x*exchange_rate_dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate_dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate_dict['INR'],2) for x in df['MC_USD_Billion']]

    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    output_statement = pd.read_sql(query_statement, sql_connection)
    print(output_statement)

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, path)
print(df['MC_EUR_Billion'][4])

log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, output_path)
log_progress('Data saved to CSV file')

log_progress('SQL Connection initiated.')
sql_connection = sqlite3.connect(db_name)

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = 'SELECT * FROM Largest_banks'
run_query(query_statement, sql_connection)

query_statement = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
run_query(query_statement, sql_connection)

query_statement = 'SELECT Name from Largest_banks LIMIT 5'
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

146.86
SELECT * FROM Largest_banks
             Name  MC_USD_Billion  MC_GBP_Billion  MC_EUR_Billion  \
0   United States          432.92          346.34          402.62   
1   United States          231.52          185.22          215.31   
2           China          194.56          155.65          180.94   
3           China          160.68          128.54          149.43   
4           India          157.91          126.33          146.86   
5   United States          155.87          124.70          144.96   
6  United Kingdom          148.90          119.12          138.48   
7   United States          140.83          112.66          130.97   
8           China          139.82          111.86          130.03   
9           China          136.81          109.45          127.23   

   MC_INR_Billion  
0        35910.71  
1        19204.58  
2        16138.75  
3        13328.41  
4        13098.63  
5        12929.42  
6        12351.26  
7        11681.85  
8        11598.07  
9    